In [29]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
data = pd.read_csv('World Football Results 2018 to 2022 - data.csv')

In [ ]:
data.info()

In [10]:
data[data['tournament']=='Southeast Asian Championship']

,year,date,tournament,h_team,a_team,h_elo,a_elo,h_score,a_score,location,neutral
708,2018,November 8,Southeast Asian Championship,Cambodia,Malaysia,796,1069,0,1,in Cambodia,No
709,2018,November 8,Southeast Asian Championship,Laos,Vietnam,730,1285,0,3,in Laos,No
710,2018,November 9,Southeast Asian Championship,Singapore,Indonesia,1068,1238,1,0,in Singapore,No
711,2018,November 9,Southeast Asian Championship,Thailand,East Timor,1398,652,7,0,in Thailand,No
714,2018,November 12,Southeast Asian Championship,Malaysia,Laos,1080,725,3,1,in Malaysia,No
715,2018,November 12,Southeast Asian Championship,Myanmar,Cambodia,1057,785,4,1,in Myanmar,No
717,2018,November 13,Southeast Asian Championship,Indonesia,East Timor,1214,651,3,1,in Indonesia,No
718,2018,November 13,Southeast Asian Championship,Philippines,Singapore,1186,1092,1,0,in the Philippines,No
752,2018,November 16,Southeast Asian Championship,Laos,Myanmar,721,1064,1,3,in Laos,No
764,2018,November 16,Southeast Asian Championship,Vietnam,Malaysia,1290,1084,2,0,in Vietnam,No


In [3]:
data = data.dropna(axis=1)

In [4]:
data.isnull().sum()

year          0
date          0
tournament    0
h_team        0
a_team        0
h_elo         0
a_elo         0
h_score       0
a_score       0
location      0
neutral       0
dtype: int64

In [ ]:
data

In [6]:
data_home = data[['h_team', 'h_score', 'a_score']]
data_away = data[['a_team', 'h_score', 'a_score']]

In [25]:
data_home = data_home.rename(columns={'h_team': 'Team', 'h_score': 'GoalsScored', 'a_score': 'GoalsConceded'})
data_away = data_away.rename(columns={'a_team': 'Team', 'h_score': 'GoalsConceded', 'a_score': 'GoalsScored'})

In [28]:
team_strength = pd.concat([data_home, data_away], ignore_index=True).groupby('Team').mean()
team_strength

,GoalsScored,GoalsConceded
Team,,
Afghanistan,0.947368,1.947368
Albania,1.066667,1.200000
Algeria,2.018868,0.849057
Andorra,0.488889,1.711111
Angola,1.205882,0.735294
...,...,...
Vietnam,1.710526,1.026316
Wales,1.000000,1.098039
Yemen,0.454545,2.090909


In [36]:
def predict_point(home, away):
    if home in team_strength.index and away in team_strength.index:
        lamb_home = team_strength.at[home, 'GoalsScored'] * team_strength.at[away, 'GoalsConceded']
        lamb_away = team_strength.at[away, 'GoalsScored'] * team_strength.at[home, 'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0 , 0
        for x in range(0,11): #number of goals home team
            for y in range(0,11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return[(points_home, points_away), (prob_home*100, prob_draw*100, prob_away*100)]
    else:
        return [(0,0), (prob_home*100, prob_draw*100, prob_away*100)]

In [37]:
predict_point('Vietnam', 'Singapore')

[(2.1109892380372846, 0.7216473605554833),
 (64.80390629075727, 16.687204931456627, 18.492510374697236)]

Vietnam wins the match.
Result: Viet Nam: 2.11 pts, Singapore: 0.72 pts


Vietnam win probability: Viet Nam: 64.8%, Draw: 16.7%, Singapore: 18.5%